In [3]:
import json
import logging
import io
import boto3
import pandas as pd
import openpyxl
import spacy
import sklearn
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

logger = logging.getLogger()
logger.setLevel(logging.INFO)

#Modify pandas settings so print statements for debugging will not be truncated
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.expand_frame_repr", True)
pd.set_option('display.width', 1000)

# Specify the paths to your .xlsx files
file_name = 'file_keys.json'

with open(file_name, 'r') as f:
    file_keys = json.load(f)
    
bucket_name = file_keys['bucket_name']
file_key_radcom = file_keys['file_key_radcom']
file_key_boost = file_keys['file_key_boost']

def file_downloader(bucket_name, file_key_attribute):
    '''Download the file into an object from the bucket name and file key'''
    s3 = boto3.client('s3')
    #download the files to a temp location
    obj = s3.get_object(Bucket=bucket_name,Key=file_key_attribute)  
    body = obj['Body'].read()
    logging.info("Body and obj loaded in")
    return body
def sheet_name_grabber(iofile):
    '''Read the bytes file into a spreadsheet and extract the sheet names'''
    attribute_ex = openpyxl.load_workbook(io.BytesIO(iofile),data_only=True)
    logging.info("The workbook has been loaded into attribute_ex")
    sheet_names_attribute = attribute_ex.sheetnames
    logging.info(f"The list of sheet names is: {sheet_names_attribute}")
    return attribute_ex, sheet_names_attribute

def df_maker(workbook, sheetnamelist):
    '''Create a list of dataframes, 1 dataframe for each "Table" in the spreadsheet'''
    #Create a list for the titles and a list for the tags
    titleslist = []
    tagslist = []
    dflist=[]
    for sheet_name in sheetnamelist:
            if 'Table' in sheet_name:
                ws = workbook[sheet_name]
                sheet_name = pd.DataFrame(ws.values)
                dflist.append(sheet_name)
    return dflist

#NOTE: The below functions each operate on 1 dataframe, not a list of dataframes
def df_header_fixer(df):
    '''Take a dataframe, move the column names to their appropriate location'''
    #Fix Column Names
    df = df.rename(columns = df.iloc[2]).loc[3:]
    return df
def df_trunkater(df):
    '''Eliminate empty rows and fix indices for a dataframe'''
    df = df[df['Attribute_Name'].notna()]
    df.index = np.arange(len(df))
    return df
def desc_filler(df):
    '''fill empty description cells'''    
    df = df[df['Attribute_Name'].notna()]
    df['Description'].fillna(df['Attribute_Name'], inplace=True)
    return df


In [4]:
'''Creating a tokenizer.  This function will take a description cell as input and processes it into tokens, performing lemmatization, lowercasing, and removing stop words.'''
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.base import TransformerMixin

# Create our list of stopwords
nlp = spacy.load("en_core_web_md")

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)
    
    # Lemmatizing each token and converting each token into lowercase
    for word in mytokens:
        if word.lemma_ != "-PRON-":
            word.lemma_.lower().strip()
        else:
            word = word.lower_    
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in STOP_WORDS and word.is_punct != True ]

    # return preprocessed list of tokens
    return mytokens

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

def cleaner(df):
    df['Cleaned_Text'] = df['Description'].apply(clean_text)
    #display(df)
    return df

tfidf_vector = sklearn.feature_extraction.text.TfidfVectorizer()


def vectorizer(df): #new, different results!
    '''Function vectorizes the "Cleaned_Text" column of the dataframe, and returns the vector'''
#    tfidf_vector = sklearn.feature_extraction.text.TfidfVectorizer(tokenizer = spacy_tokenizer, token_pattern='')
    X = df['Cleaned_Text']
    #tvectorizer = TfidfVectorizer()
    X = tfidf_vector.fit_transform(X)
    #tvectorizer.get_feature_names_out()
    return X


def bestwordsfinder(X):
    '''Function takes a vector of tfidf scores and returns the words with tfidf values >=threshold, and their scores'''
    feature_names = tfidf_vector.get_feature_names_out()
    goodwords = []
    goodvals = []
    #threshold = 0
    for row in range(X.shape[0]):  # Iterate over each row in the matrix
        for col in X[row, :].nonzero()[1]:  # Get the indices of non-zero elements in the row
            word = feature_names[col]
            if word not in goodwords:
                goodwords.append(word)
                goodvals.append(X[row,col])
          
    tagsdf = pd.DataFrame(data = {'Word': goodwords, 'tf-idf_score': goodvals})
    # score_percentile = tagsdf['tf-idf_score'].quantile(0.95)
    # tagsdf = tagsdf[tagsdf['tf-idf_score'] > score_percentile]
    tagsdf = tagsdf.nlargest(6, 'tf-idf_score')

    
    return tagsdf

In [5]:
def import_pipeline(bucket, file_key):
    '''Function to pipeline the import process, beginning with an s3 file location, and eventually outputting a list of data frames, one for each table in the spreadsheet'''
    body = file_downloader(bucket, file_key)
    workbook, sheet_names= sheet_name_grabber(body)
    dflist = df_maker(workbook, sheet_names)
    return dflist
def pipeline(dataframelist, functions):
    '''Takes in a dataframe and returns word vectors'''
    results = []
    for df in dataframelist:
        processed_df = df.copy()  # Create a copy of the original DataFrame
        for func in functions:
            processed_df = func(processed_df)  # Apply each function in sequence
        results.append(processed_df)
    
    return results

In [6]:
functionsall = [df_header_fixer, df_trunkater, desc_filler, cleaner, vectorizer, bestwordsfinder] #, tokenizer
functionsvec = [df_header_fixer, df_trunkater, desc_filler, cleaner, vectorizer] #, tokenizer
functions = [df_header_fixer, df_trunkater, desc_filler, cleaner] #, tokenizer 

# boostdf = pipeline(import_pipeline(bucket_name, file_key_boost), functions) #[0]['Tokenized_List']
# radcomdf = pipeline(import_pipeline(bucket_name, file_key_radcom), functions) #[0]['Tokenized_List']

# boostdf = pipeline(import_pipeline(bucket_name, file_key_boost), functionsvec) #[0]['Tokenized_List']
# radcomdf = pipeline(import_pipeline(bucket_name, file_key_radcom), functionsvec) #[0]['Tokenized_List']

boostdf = pipeline(import_pipeline(bucket_name, file_key_boost), functionsall) #[0]['Tokenized_List']
radcomdf = pipeline(import_pipeline(bucket_name, file_key_radcom), functionsall) #[0]['Tokenized_List']


#X = pipeline(dflist, functions)[0] #[0]['Tokenized_List']
print('Boost')
for i in range(len(boostdf)):
    display(boostdf[i])
print(f'\n')    
print('Radcom')
for i in range(len(radcomdf)):
    display(radcomdf[i])

INFO:root:Body and obj loaded in
INFO:root:The workbook has been loaded into attribute_ex
INFO:root:The list of sheet names is: ['Instructions', 'Table1', 'Table2']
INFO:root:Body and obj loaded in
INFO:root:The workbook has been loaded into attribute_ex
INFO:root:The list of sheet names is: ['Instructions', 'Table1', 'Table2', 'Table3', 'Table4', 'Table5']


Boost


,Word,tf-idf_score
234,padded,0.878047
230,country,0.744706
217,tech,0.674363
101,gsm,0.666215
93,activity,0.651609
201,direction,0.621861


,Word,tf-idf_score
280,padded,0.866420
137,00,0.694314
106,gsm,0.693110
275,id,0.651783
192,cycle,0.624126
279,allocation,0.597896




Radcom


,Word,tf-idf_score
114,diversion,1.000000
71,initial,0.841023
0,alias,0.707107
1,src,0.707107
186,count,0.707107
187,msg,0.707107


,Word,tf-idf_score
90,nssai_sst,1.000000
104,resource,1.000000
113,supi,1.000000
114,suci,1.000000
105,stream,0.804975
101,operation,0.742203


,Word,tf-idf_score
82,supi,1.000000
83,suci,1.000000
121,count,0.707107
122,frames,0.707107
74,ue,0.641995
128,5gmm,0.634042


,Word,tf-idf_score
198,website,1.000000
214,source,0.630501
94,address,0.620954
95,ip,0.620954
183,mme,0.607900
226,denotes,0.598321


,Word,tf-idf_score
161,authenticate,1.0
177,imei,1.0
195,kasme,1.0
199,manufacturer,1.0
200,model,1.0
221,impu,1.0
